In [ ]:
www.tinyurl.com/aidd-gnina

**Molecular docking and scoring using Gnina.**

References: McNutt, A.T., Francoeur, P., Aggarwal, R. et al. GNINA 1.0: molecular docking with deep learning. J Cheminform 13, 43 (2021). https://doi.org/10.1186/s13321-021-00522-2

Github Link: https://github.com/gnina/gnina


- **Gnina** is a molecular docking software that builds upon **AutoDock Vina** with added deep learning capabilities.
- It integrates **convolutional neural networks (CNNs)** to enhance the scoring of protein–ligand interactions.
- Performs traditional docking to generate ligand binding poses within the receptor's active site.
- **CNN-based scoring functions** analyze 3D molecular interaction patterns, offering improved pose prediction and binding affinity estimation.
- Supports **flexible ligand docking** and can be accelerated using **GPUs**, improving performance and scalability.
- Provides multiple scores per pose:
  - **Vina score** (classical scoring)
  - **CNN pose score** (likelihood of correct pose)
  - **CNN affinity score** (predicted binding affinity)
- **Advantages**:
  - More accurate pose ranking compared to classical methods.
  - Effective for **virtual screening** and **structure-based drug design**.
  - Handles diverse ligand chemistries and binding site environments.


In [ ]:
# Install the py3Dmol tool for visualization of the protein structure.
!pip install py3Dmol rdkit

In [ ]:
!apt install openbabel

In [ ]:
# Since it uses GPU for scoring make sure that you have selected GPU in Runtime. If not go to Runtime and select Change runtime Type and select GPU.
!nvidia-smi

In [ ]:
!mkdir Ache


In [ ]:
cd Ache

**Things to consider when selecting a PDB file as a receptor**
1. Resolution: Choose a structure with high resolution (typically <2.5 Å); lower values mean better-quality, more reliable atomic positions.
2. Biological relevance: Make sure the PDB structure represents the biologically active form (correct chain, quaternary structure, oligomerization state, etc.).
3. Presence of ligands/cofactors: Prefer structures that already have a bound ligand if available — it helps identify the active site and correct conformation.
4. Mutations or modifications: Check for engineered mutations, missing residues, or post-translational modifications that could affect binding.
5. Missing atoms or residues: Avoid structures with major missing parts around the binding site unless you plan to model or fix them.
6. Water molecules: Decide whether crystallographic water molecules are needed (some near active sites can be critical, most are not).
7. Chain selection: Choose the correct chain if multiple chains are present; avoid selecting unintended parts like symmetry mates.
8. Redundancy: If multiple PDBs are available for the same target, select the one with the best quality, resolution, and binding site representation.
9. Experimental method: Prefer X-ray or cryo-EM structures over low-quality NMR models unless no better data exist.
10. Relevance to your ligand: Ideally, pick a receptor structure co-crystallized with a ligand similar to your docking molecule to ensure a realistic binding site conformation.

In [ ]:
!wget http://files.rcsb.org/download/1EVE.pdb

In [ ]:
import py3Dmol

# Load the full PDB structure
with open('1EVE.pdb', 'r') as f:
    pdb_data = f.read()
# Create the viewer
view = py3Dmol.view(width=1080, height=720)
view.addModel(pdb_data, 'pdb')
# Style the protein
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'spectrum'}})
# Highlight the ligand E20 in stick style
view.setStyle({'resn': 'E20'}, {'stick': {'radius': 0.25, 'color': 'magenta'}})
# Zoom to the ligand region
view.zoomTo({'resn': 'E20'})
# Show the viewer
view.show()


In [ ]:
!ls

**Steps to prepare receptor.**
1. Obtain the Receptor Structure: Download the receptor's PDB file from the Protein Data Bank (PDB).
2. Remove Water and Non-Relevant Molecules: Filter out water molecules and other non-receptor molecules using grep or similar tools.
3. Add Hydrogens: Use tools like Open Babel or Reduce to add missing hydrogens to the receptor structure.
4. Assign Protonation States: Automatically assign the correct protonation states for ionizable residues using Open Babel or Reduce.
5. Fix Missing/Incorrect Atom Types: Correct missing or incorrect atom types during hydrogen addition with tools like Open Babel or Reduce.
6. Prepare for Docking Format (PDBQT): Convert the PDB file into a PDBQT file, adding partial charges and atom types using AutoDockTools or similar tools.
7. Define the Docking Grid: Set up the docking grid box centered around the receptor's binding site using tools like ProDy or PyMOL.
8. Final Preparation Check: Verify the receptor structure and docking parameters to ensure they are ready for simulation.

(Use Meeko for proper preparation of the receptor file https://meeko.readthedocs.io/en/release/lig_prep_basic.html)

In [ ]:
!grep ATOM 1EVE.pdb > rec.pdb
!obabel rec.pdb -O prepared.pdb

In [ ]:
!grep E20 1EVE.pdb > lig.pdb


**Steps to prepare ligands**
1. Obtain ligand structure: Acquire the ligand from a database or design it, ensuring it’s in a compatible format (PDB, MOL, SDF).
2. Remove non-ligand components: Remove water, heteroatoms, and other non-ligand entities from the structure.
3. Add hydrogens: Use tools like Open Babel to add missing hydrogen atoms to the ligand structure.
Assign partial charges: Assign partial charges (e.g., Gasteiger) to atoms to represent the ligand’s electronic distribution.
4. Minimize the structure: Perform energy minimization to relax the ligand’s geometry and remove steric clashes.
5. Check tautomers and ionization state: Ensure the ligand is in the correct tautomeric form and ionization state, generating possible alternatives if needed.
6. Convert to docking format: Convert the ligand to the required input format for docking tools (e.g., PDBQT for AutoDock).
7. Visualize and clean the structure: Inspect the ligand for issues like steric clashes and correct geometry using molecular visualization tools.

Use Meeko for proper ligand preparation. (https://meeko.readthedocs.io/en/release/lig_overview.html)

In [ ]:
!obabel lig.pdb -O lig_optimized.sdf --addhydrogens --minimize --ff UFF --partialcharge gasteiger


In [ ]:
import py3Dmol

# Create a 3Dmol view object
view = py3Dmol.view(width=800, height=600)

# Load the optimized ligand (SDF file)
with open('lig_optimized.sdf', 'r') as f:
    optimized_data = f.read()
view.addModel(optimized_data, 'sdf')
view.setStyle({'model': 0}, {'stick': {'colorscheme': 'cyanCarbon', 'radius': 0.15}})

# Load the original ligand (PDB file)
with open('lig.pdb', 'r') as f:
    original_data = f.read()
view.addModel(original_data, 'pdb')
view.setStyle({'model': 1}, {'stick': {'colorscheme': 'greenCarbon', 'radius': 0.15}})

# Align and zoom
view.zoomTo()

# Show both structures
view.show()



In [ ]:
!wget https://github.com/gnina/gnina/releases/download/v1.0.1/gnina

In [ ]:
!chmod +x gnina

In [ ]:
!./gnina --version

In [ ]:
!./gnina -help

In [ ]:
!ls

In [ ]:
!./gnina -r prepared.pdb -l lig_optimized.sdf --autobox_ligand lig.pdb -o docked.sdf --seed 0

In [ ]:
import py3Dmol
v = py3Dmol.view(width=1080, height=720)


chA = {'chain':'A'}
v.addModel(open('rec.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.setStyle({'cartoon':{'color': 'spectrum'}})
#v.addSurface(py3Dmol.VDW,{'opacity':0.95,'color':'white'}, chA)
v.addModel(open('lig.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
v.addModelsAsFrames(open('docked.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.animate({'interval':1000})
v.zoomTo({'model':1})
v.rotate(90)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

In [ ]:
!zip -r Ache.zip docked.sdf prepared.pdb lig_optimized.sdf 1EVE.pdb
from google.colab import files
files.download("/content/Ache/Ache.zip")